In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import gensim.downloader as api

In [2]:
# Load dataset
file_path = "/Users/celinewu/Documents/GitHub/2024-25c-fai2-adsai-group-group16/Task_4/ver_2_FINAL_DATASET.xlsx"
df = pd.read_excel(file_path)

In [3]:
# Extract text and labels
sentences = df["Sentence"].astype(str).tolist()
labels = df["main_category"].astype(str).tolist()

# Extract Sentiment Scores
sentiment_scores = df["Sentiment_Score"].values.reshape(-1, 1)

# Normalize Sentiment Scores
scaler = MinMaxScaler()
sentiment_scores = scaler.fit_transform(sentiment_scores)


In [4]:
# Tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(sentences)
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding="post")


In [5]:
# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)


In [6]:
# Split dataset
X_train, X_test, y_train, y_test, X_train_sent, X_test_sent = train_test_split(
    padded_sequences, encoded_labels, sentiment_scores, 
    test_size=0.2, random_state=42, stratify=encoded_labels
)

In [7]:
# Convert labels to categorical
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)


In [8]:
# Load GloVe embeddings
print("Loading GloVe embeddings...")
glove_model = api.load("glove-wiki-gigaword-100")  # 100D embeddings

# Create embedding matrix
embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in glove_model:
        embedding_matrix[i] = glove_model[word]

# Define text input and embedding layer (using pretrained embeddings)
text_input = Input(shape=(max_length,), name="text_input")
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, 
                      weights=[embedding_matrix], input_length=max_length, 
                      trainable=False)(text_input)


Loading GloVe embeddings...
[==================================================] 100.0% 128.1/128.1MB downloaded


/Users/celinewu/anaconda3/envs/2B/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-03-03 13:38:08.345466: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2025-03-03 13:38:08.345511: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-03-03 13:38:08.345516: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-03-03 13:38:08.345754: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-03 13:38:08.345768: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefin

In [9]:
# Define GRU layers
rnn_layer = GRU(128, return_sequences=True, recurrent_dropout=0.3)(embedding)
rnn_layer = GRU(64, recurrent_dropout=0.3)(rnn_layer)
dense_text = Dense(64, activation='relu')(rnn_layer)


In [10]:
# Define Sentiment Score input
sentiment_input = Input(shape=(1,), name="sentiment_input")
sentiment_dense = Dense(8, activation='relu')(sentiment_input)

# Merge both inputs
merged = Concatenate()([dense_text, sentiment_dense])
output = Dense(num_classes, activation='sigmoid')(merged)  # Sigmoid for multi-emotion


In [11]:
# Define final model
model = Model(inputs=[text_input, sentiment_input], outputs=output)
optimizer = Adam(learning_rate=0.0005)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [12]:
# Compute class weights to handle imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(encoded_labels), y=encoded_labels)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}


In [13]:
# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint("rnn_sentiment_score_it2.keras", monitor='val_accuracy', save_best_only=True)

# Train the model
print("Training model...")
history = model.fit(
    [X_train, X_train_sent], y_train,
    epochs=25, batch_size=32, validation_data=([X_test, X_test_sent], y_test),
    class_weight=class_weight_dict,
    callbacks=[early_stopping, model_checkpoint]
)

Training model...
Epoch 1/25


2025-03-03 13:38:09.340875: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


140/140 ━━━━━━━━━━━━━━━━━━━━ 391s 3s/step - accuracy: 0.1299 - loss: 63.1954 - val_accuracy: 0.1281 - val_loss: 1.9472
Epoch 2/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 387s 3s/step - accuracy: 0.1425 - loss: 2.0298 - val_accuracy: 0.1676 - val_loss: 1.9428
Epoch 3/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 384s 3s/step - accuracy: 0.1637 - loss: 1.9605 - val_accuracy: 0.2186 - val_loss: 1.9396
Epoch 4/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 4007s 29s/step - accuracy: 0.1814 - loss: 1.9620 - val_accuracy: 0.2240 - val_loss: 1.9365
Epoch 5/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 1749s 13s/step - accuracy: 0.1770 - loss: 1.9736 - val_accuracy: 0.2231 - val_loss: 1.9333
Epoch 6/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 568s 4s/step - accuracy: 0.2055 - loss: 1.9949 - val_accuracy: 0.2177 - val_loss: 1.9302
Epoch 7/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 430s 3s/step - accuracy: 0.2059 - loss: 1.9575 - val_accuracy: 0.2115 - val_loss: 1.9270
Epoch 8/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 2873s 21s/step - accuracy: 0.2058 - loss: 1.9316 - val_accuracy

In [14]:
# Evaluate the model
test_loss, test_acc = model.evaluate([X_test, X_test_sent], y_test)
print(f"Test Accuracy: {test_acc:.4f}")


35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step - accuracy: 0.2248 - loss: 1.8811
Test Accuracy: 0.2142


In [15]:
# Save tokenizer
import pickle
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)


In [16]:
# Generate F1-score report
from sklearn.metrics import classification_report

# Get predictions
y_pred = model.predict([X_test, X_test_sent])

# Convert predictions from one-hot encoding to class indices
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Generate classification report
print("\nClassification Report:")
print(classification_report(y_test_classes, y_pred_classes, target_names=label_encoder.classes_))

35/35 ━━━━━━━━━━━━━━━━━━━━ 5s 149ms/step

Classification Report:
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00       159
     disgust       0.28      0.49      0.36       159
        fear       0.00      0.00      0.00       159
   happiness       0.19      0.89      0.32       160
     neutral       0.00      0.00      0.00       160
     sadness       0.20      0.12      0.15       159
    surprise       0.00      0.00      0.00       160

    accuracy                           0.21      1116
   macro avg       0.10      0.21      0.12      1116
weighted avg       0.10      0.21      0.12      1116



/Users/celinewu/anaconda3/envs/2B/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/celinewu/anaconda3/envs/2B/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/celinewu/anaconda3/envs/2B/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()

In [17]:
# Count occurrences of each unique value in the 'main_category' column
emotion_counts = df["main_category"].value_counts()

# Display the result
print(emotion_counts)

main_category
happiness    797
disgust      797
anger        797
surprise     797
sadness      797
fear         797
neutral      797
Name: count, dtype: int64
